In [7]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset, concatenate_datasets

def format_for_belar(row):
    row["context"] = row["selftext"]
    row["question"] = row["title"]
    row['answers'] = row["answers"]["text"]
    return row
    
d = load_dataset("eli5")
ds = d['test_eli5'].map(format_for_belar, batched=False)
ds = ds.select_columns(["context", "question", "answers"])

Found cached dataset eli5 (/home/jjmachan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


  0%|          | 0/9 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jjmachan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-b6864f61633d1e41.arrow


In [3]:
ds = ds.shuffle().select(range(100))
ds.shape

(100, 3)

In [6]:
def get_answers(row):
    row["answer_generated"] = llm(prompt.format(
        context=row['context'],
        question=row['question'])
       )
    return row
    
ds = ds.map(get_answers, batched=False)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## Evalutate

In [8]:
from belar.metrics import Rouge1, Evaluation

In [9]:
ds.column_names

['context', 'question', 'answers', 'answer_generated']

In [15]:
def rename(row):
    row["prompt"] = row["question"]
    row["references"] = []
    row["ground_truth"] = row["answers"]
    row["generated_text"] = row["answer_generated"]
    
    return row

ds = ds.map(rename).select_columns(
    ["prompt", "context", "references", "ground_truth", "generated_text"]
)
ds

Dataset({
    features: ['context', 'prompt', 'references', 'ground_truth', 'generated_text'],
    num_rows: 100
})

In [39]:
e = Evaluation(metrics=[Rouge1])
e.eval(ds.select_columns(["ground_truth"]),
                         ds["generated_text"])

[[0.3235294117647059,
  0.3870967741935484,
  0.3372093023255814,
  0.296875,
  0.3785488958990536,
  0.19729206963249518,
  0.27807486631016043,
  0.25225225225225223,
  0.15196078431372548,
  0.1170212765957447,
  0.24509803921568626,
  0.1894736842105263,
  0.3106796116504854,
  0.3240223463687151,
  0.20689655172413796,
  0.15686274509803924,
  0.11363636363636362,
  0.27636363636363637,
  0.2524271844660194,
  0.380952380952381,
  0.233502538071066,
  0.1842105263157895,
  0.22900763358778622,
  0.1518987341772152,
  0.2631578947368421,
  0.3764705882352941,
  0.2301369863013699,
  0.2553191489361702,
  0.20689655172413796,
  0.20930232558139533,
  0.4035087719298246,
  0.24295774647887325,
  0.271604938271605,
  0.22307692307692306,
  0.18181818181818182,
  0.37818181818181823,
  0.15436241610738255,
  0.17500000000000002,
  0.30075187969924805,
  0.1257142857142857,
  0.3574144486692016,
  0.20408163265306123,
  0.36601307189542487,
  0.2637362637362637,
  0.21794871794871798,
 

In [36]:
type(ds["generated_text"])

list